In [1]:
from io import StringIO
import numpy as np
# import polars as pl
import pandas as pd
from subprocess import call
# from gtfparse import read_gtf

### Set parameters

In [2]:
exon_flank_nt = 5 # flanking nucleotides from the start and end of exons
number_of_threads = 4 # number of threads used in bcftools output compression
tag_str = 'Tian_080423' # DNAnexus job tag

project_path = 'project-GGy3Bb0JqBj7zfxY8v4by61X:/'

# DNAnexus input vcf path (end with '/')
dx_vcf_path = project_path + "Bulk/Exome\ sequences/Population\ level\ exome\ OQFE\ variants,\ pVCF\ format\ -\ final\ release/"
# DNAnexus gene-specific vcf output path (end with '/')
dx_vcf_out_path = project_path + "Tian_folder/14gene_vcf/"
# DNAnexus heterogeneous carrier output path (under dx_vcf_out_path)
dx_carrier_out_path = "hetero_carrier/"
# DNAnexus resource path (end with '/')
dx_resource_path = project_path + "GRCh38_resources/"
# DNAnexus script path (end with '/')
dx_script_path = project_path + "scripts/"
# difficult regions bed file (under dx_resource_path)
diff_bed = 'GRCh38_alldifficultregions.bed.gz'
# reference genome file (under dx_resource_path)
ref_genome = 'GRCh38_reference_genome.fa' # index filename is inferred

!dx mkdir -p {dx_vcf_out_path} # create gene.vcf output folder

### Helper functions

In [3]:
def get_overlapping_UKB_vcfs(df_gene, df_blk):
    vcf_prefix = 'ukb23157_'
    vcf_suffix = '_v1.vcf.gz'
    
    vcf_files = []
    for index, row in df_gene.iterrows():
        vcf_files = vcf_files + df_blk.loc[(df_blk['seqname'] == row['seqname']) & 
                                           # when the gene is completely contained inside the block
                                           (((df_blk['start_pos'] <= row['exon_flank_start']) & (df_blk['end_pos'] >= row['exon_flank_end'])) |
                                            # when the block is completely contained inside the gene
                                            ((df_blk['start_pos'] >= row['exon_flank_start']) & (df_blk['end_pos'] <= row['exon_flank_end'])) |
                                            # when the gene overlaps with the start of the block
                                            (df_blk['start_pos'].between(row['exon_flank_start'], row['exon_flank_end'])) |
                                            # when the gene overlaps with the end of the block
                                            (df_blk['end_pos'].between(row['exon_flank_start'], row['exon_flank_end']))),
                                           'chr_blk_str'].tolist()
    vcf_files = set(vcf_prefix + x + vcf_suffix for x in set(vcf_files))
    return(vcf_files)

### List of gene symbols as input

In [4]:
genes = ['LDLRAP1', 'PSMD4', 'GPN2', 'RABIF', 'RAB10', 'PREB', 'HMGCR', 'MYLIP', 'NPC1L1', 'ABCA1', 'HNF1A', 'OTX2', 'HNF4A', 'SOD1']

# # to read from txt:
# with open("resources/my_gene_file.txt", "r") as gene_file:
#     genes = gene_file.readlines()
#     genes = [l.replace("\n", "") for l in genes]
#     print(genes)

### Load pVCF block coordinates

In [5]:
df_blk = pd.read_table("./resources/pvcf_blocks.txt", sep = '\t', names = ['ind', 'chr', 'blk', 'start_pos', 'end_pos'])
df_blk['seqname'] = 'chr' + df_blk['chr'].map(str)
df_blk.loc[df_blk['seqname'] == 'chr23', 'seqname'] = 'chrX'
df_blk.loc[df_blk['seqname'] == 'chr24', 'seqname'] = 'chrY'
df_blk['chr_blk_str'] = df_blk['seqname'].str.replace('chr', 'c') + '_b' + df_blk['blk'].map(str)
df_blk

,ind,chr,blk,start_pos,end_pos,seqname,chr_blk_str
0,1,1,0,1,1218130,chr1,c1_b0
1,2,1,1,1218131,1426969,chr1,c1_b1
2,3,1,2,1426970,1758871,chr1,c1_b2
3,4,1,3,1758872,2514221,chr1,c1_b3
4,5,1,4,2514222,3782130,chr1,c1_b4
...,...,...,...,...,...,...,...
972,973,23,20,135552245,141897932,chrX,cX_b20
973,974,23,21,141897933,152168662,chrX,cX_b21
974,975,23,22,152168663,153788223,chrX,cX_b22
975,976,23,23,153788224,156040895,chrX,cX_b23


### Load MANE transcript coordinates

In [6]:
# df = read_gtf("./resources/MANE.GRCh38.v1.0.select_ensembl_genomic.gtf.gz")
# df = df.filter(pl.col('feature') == 'exon')
# df = df.filter(pl.col('gene_name').is_in(genes))
# df = df.select(pl.col(['seqname', 'start', 'end', 'gene_name']))
# df = df.with_columns([(pl.col('start') - exon_flank_nt).alias('exon_flank_start'), 
#                       (pl.col('end') + exon_flank_nt).alias('exon_flank_end')])
# df = df.with_column((pl.col('seqname')+':'+pl.col('exon_flank_start')+'-'+pl.col('exon_flank_end')).alias('region'))
# df

df = pd.read_csv("./resources/MANE.GRCh38.v1.0.select_ensembl_genomic.csv.gz")
df = df.loc[(df['feature'] == 'exon') & (df['gene_name'].isin(genes))]
df = df[['seqname', 'start', 'end', 'gene_name']]
df['exon_flank_start'] = df['start'] - exon_flank_nt
df['exon_flank_end'] = df['end'] + exon_flank_nt
df['region'] = ((df['seqname'] + ':').str.cat(df['exon_flank_start'].astype(str)) + '-').str.cat(df['exon_flank_end'].astype(str))
df

,seqname,start,end,gene_name,exon_flank_start,exon_flank_end,region
1476,chr1,25543606,25543786,LDLRAP1,25543601,25543791,chr1:25543601-25543791
1479,chr1,25553922,25554064,LDLRAP1,25553917,25554069,chr1:25553917-25554069
1481,chr1,25554860,25554972,LDLRAP1,25554855,25554977,chr1:25554855-25554977
1483,chr1,25557153,25557267,LDLRAP1,25557148,25557272,chr1:25557148-25557272
1485,chr1,25562644,25562716,LDLRAP1,25562639,25562721,chr1:25562639-25562721
...,...,...,...,...,...,...,...
486568,chr21,31659693,31659841,SOD1,31659688,31659846,chr21:31659688-31659846
486571,chr21,31663790,31663886,SOD1,31663785,31663891,chr21:31663785-31663891
486573,chr21,31666449,31666518,SOD1,31666444,31666523,chr21:31666444-31666523
486575,chr21,31667258,31667375,SOD1,31667253,31667380,chr21:31667253-31667380


### Run Swiss-army-knife on DNAnexus
- get region info for each gene from block file
- bcftools command for step 2 finished
- bcftools command for step 3 & 4 TBD
- list genes not included in MANE set

In [7]:
genes_not_found = []
genes_found = []
known_large_genes = ["DSP", "TSC2", "TTN", "NCOA3"]

for gene in genes:
#     df_gene = df.filter(pl.col('gene_name') == gene)
    df_gene = df.loc[df['gene_name'] == gene]
    if df_gene.shape[0] > 0:
        genes_found.append(gene)
        
        vcf_files = get_overlapping_UKB_vcfs(df_gene, df_blk)
        vcf_str = ' '.join(vcf_files)
        region_str = ','.join(df_gene['region'].to_list())
        
        if ((len(vcf_files) > 1) or (gene in known_large_genes)):
            mem_level = "mem2_ssd1_v2_x16" # dynamically change memory level when submitting jobs on DNAnexus
        else:
            mem_level = "mem1_ssd1_v2_x4"
        
        # filtering commands
        bcftools_cmd1 = "bcftools concat -Ou -a -r " + region_str + " " + vcf_str
        bcftools_cmd2 = "bcftools view -Ou --max-alleles 5 -T ^" + diff_bed
        bcftools_cmd3 = "bcftools norm -Ou -m - -f " + ref_genome
        bcftools_cmd4 = "bcftools annotate -Ou --set-id '%CHROM\_%POS\_%REF\_%ALT'"
        bcftools_cmd5 = "bcftools view -Oz -i 'AF<=0.001 && MAC >=1 && F_MISSING<0.1 && F_PASS(DP>=10 & GT!=\\\"mis\\\")> 0.9' > " + gene + "_variants.vcf.gz"
        
        # variant-only vcf command 
        bcftools_command = " | ".join([bcftools_cmd1, bcftools_cmd2, bcftools_cmd3, bcftools_cmd4, bcftools_cmd5])
        
        # get genotypes
        genotype_cmd0 = "mkdir -p " + dx_carrier_out_path
        genotype_cmd1 = "bcftools query -i 'GT=\\\"RA\\\"|GT=\\\"AR\\\"' -f '%CHROM  %POS %REF %ALT %INFO/AF [%SAMPLE|]\n' " + gene + "_variants.vcf.gz > " + dx_carrier_out_path + gene + ".ssv"
        
        # get genotypes command
        genotype_command = " && ".join([genotype_cmd0, genotype_cmd1])

        dx_input_str = ' '.join(set('-iin="' + dx_vcf_path + x + '"' for x in vcf_files).union(set('-iin="' + dx_vcf_path + x + '.tbi"' for x in vcf_files)))
        dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + diff_bed + '"'
        dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + ref_genome + '"'
        dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + ref_genome + '.fai"'

        dx_command = 'dx run swiss-army-knife --instance-type ' + mem_level + ' -y --brief ' + dx_input_str + ' -icmd="' + bcftools_command + ' && ' + genotype_command + '" --destination ' + dx_vcf_out_path + ' --tag "' + tag_str + '" --property gene=' + gene

        !{dx_command}
    else:
        genes_not_found.append(gene)
            
print('Genes not found in MANE database:')
print(genes_not_found)

job-GY6Yx88JqBj16f1YBv2JJP99
job-GY6Yx8QJqBjFvP5vf4K7qZbv
job-GY6Yx8jJqBj61fZpBGXV8jbZ
job-GY6Yx98JqBj3FjfKbq7fqFK6
job-GY6Yx9QJqBj7zQz8xQFbk43Q
job-GY6Yx9jJqBjJpJ1YV1ppZy8p
job-GY6YxBQJqBj9031xQk349Kq6
job-GY6YxF0JqBj02FPzf62PKpj2
job-GY6YxF8JqBj1P949B6Y2kZ1p
job-GY6YxFQJqBjF697XkKbZFf9P
job-GY6YxFjJqBj23473kKQZ04kZ
job-GY6YxG8JqBj29bkz9zXgpQKF
job-GY6YxGQJqBjFvb3BvFbgXzzV
job-GY6YxGjJqBj11qkkV97xbv25
Genes not found in MANE database:
[]


In [8]:
dx_command

'dx run swiss-army-knife --instance-type mem1_ssd1_v2_x4 -y --brief -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/Bulk/Exome\\ sequences/Population\\ level\\ exome\\ OQFE\\ variants,\\ pVCF\\ format\\ -\\ final\\ release/ukb23157_c1_b36_v1.vcf.gz" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/Bulk/Exome\\ sequences/Population\\ level\\ exome\\ OQFE\\ variants,\\ pVCF\\ format\\ -\\ final\\ release/ukb23157_c1_b36_v1.vcf.gz.tbi" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/GRCh38_resources/GRCh38_alldifficultregions.bed.gz" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/GRCh38_resources/GRCh38_reference_genome.fa" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/GRCh38_resources/GRCh38_reference_genome.fa.fai" -icmd="bcftools concat -Ou -a -r chr1:55039543-55040049,chr1:55043838-55044039,chr1:55046518-55046651,chr1:55052273-55052416,chr1:55052645-55052796,chr1:55055988-55056194,chr1:55057326-55057519,chr1:55058031-55058214,chr1:55058494-55058652,chr1:55059481-55059668,chr1:55061370-55061561,chr1:55063364-55064857 ukb23

In [ ]:
dx run swiss-army-knife 
--instance-type mem1_ssd1_v2_x4 
-y --brief 
-iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/Bulk/Exome\\ sequences/Population\\ level\\ exome\\ OQFE\\ variants,\\ pVCF\\ format\\ -\\ final\\ release/ukb23157_c1_b36_v1.vcf.gz" 
-iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/Bulk/Exome\\ sequences/Population\\ level\\ exome\\ OQFE\\ variants,\\ pVCF\\ format\\ -\\ final\\ release/ukb23157_c1_b36_v1.vcf.gz.tbi" -iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/GRCh38_resources/GRCh38_alldifficultregions.bed.gz" 
-iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/GRCh38_resources/GRCh38_reference_genome.fa" 
-iin="project-GGy3Bb0JqBj7zfxY8v4by61X:/GRCh38_resources/GRCh38_reference_genome.fa.fai" 
-icmd="bcftools concat -Ou -a -r chr1:55039543-55040049,chr1:55043838-55044039,chr1:55046518-55046651,chr1:55052273-55052416,chr1:55052645-55052796,chr1:55055988-55056194,chr1:55057326-55057519,chr1:55058031-55058214,chr1:55058494-55058652,chr1:55059481-55059668,chr1:55061370-55061561,chr1:55063364-55064857 ukb23157_c1_b36_v1.vcf.gz | bcftools view -Ou --max-alleles 5 -T ^GRCh38_alldifficultregions.bed.gz | bcftools norm -Ou -m - -f GRCh38_reference_genome.fa | bcftools annotate -Ou --set-id \'%CHROM\\_%POS\\_%REF\\_%ALT\' | bcftools view -Oz -i \'AF<=0.001 && MAC >=1 && F_MISSING<0.1 && F_PASS(DP>=10 & GT!=\\"mis\\")> 0.9\' > PCSK9_variants.vcf.gz && mkdir -p hetero_carrier/ && bcftools query -i \'GT=\\"RA\\"|GT=\\"AR\\"\' -f \'%CHROM  %POS %REF %ALT %INFO/AF [%SAMPLE|]\n\' PCSK9_variants.vcf.gz > hetero_carrier/PCSK9.ssv" 
--destination project-GGy3Bb0JqBj7zfxY8v4by61X:/Vineel_genes/filtered_vcf_5/ 
--tag "Vineel_070722"
--property gene=PCSK9

### Below is for testing purposes

In [53]:
gene = 'APOB'
df_gene = df.filter(pl.col('gene_name') == gene)
vcf_files = get_overlapping_UKB_vcfs(df_gene, df_blk)
vcf_str = ' '.join(vcf_files)
region_str = ','.join(df_gene['region'].to_list())
mem_level = len(vcf_files)

bcftools_cmd1 = "bcftools concat -Ou -a -r " + region_str + " " + vcf_str
bcftools_cmd2 = "bcftools view -Ou -T ^" + diff_bed
bcftools_cmd3 = "bcftools norm -Ou -m - -f " + ref_genome
bcftools_cmd4 = "bcftools annotate -Ou --set-id '%CHROM\_%POS\_%REF\_%ALT'"
bcftools_cmd5 = "bcftools view -Oz --threads " + str(number_of_threads) + " -i 'AF<=0.001 && MAC >=1 && F_MISSING<0.1 && F_PASS(DP>=10 & GT!=\\\"mis\\\")> 0.9' > " + gene + ".vcf.gz"
bcftools_command = " | ".join([bcftools_cmd1, bcftools_cmd2, bcftools_cmd3, bcftools_cmd4, bcftools_cmd5])

dx_input_str = ' '.join(set('-iin="' + dx_vcf_path + x + '"' for x in vcf_files).union(set('-iin="' + dx_vcf_path + x + '.tbi"' for x in vcf_files)))
dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + diff_bed + '"'
dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + ref_genome + '"'
dx_input_str = dx_input_str + ' -iin="' + dx_resource_path + ref_genome + '.fai"'

dx_command = 'dx run swiss-army-knife --instance-type mem' + str(mem_level) + '_ssd1_v2_x4 -y --brief ' + dx_input_str + ' -icmd="' + bcftools_command + '" --destination ' + dx_vcf_out_path + ' --tag "' + tag_str + '" --property gene=' + gene

print(bcftools_command)
print(dx_command)



bcftools concat -Ou -a -r chr2:21043859-21044078,chr2:21043508-21043556,chr2:21042356-21042481,chr2:21040933-21041088,chr2:21037953-21038116,chr2:21037095-21037260,chr2:21035579-21035713,chr2:21034811-21034906,chr2:21033294-21033523,chr2:21032349-21032586,chr2:21029893-21030020,chr2:21029634-21029790,chr2:21028322-21028543,chr2:21027823-21028070,chr2:21026783-21026969,chr2:21024928-21025129,chr2:21023520-21023697,chr2:21022826-21023047,chr2:21019718-21019910,chr2:21018987-21019118,chr2:21016434-21016654,chr2:21015365-21015550,chr2:21015068-21015265,chr2:21014443-21014598,chr2:21013155-21013538,chr2:21005075-21012656,chr2:21004556-21004680,chr2:21004264-21004457,chr2:21001424-21003339 ukb23157_c2_b3_v1.vcf.gz ukb23157_c2_b4_v1.vcf.gz | bcftools view -Ou -T ^GRCh38_alldifficultregions.bed.gz | bcftools norm -Ou -m - -f GRCh38_reference_genome.fa | bcftools annotate -Ou --set-id '%CHROM\_%POS\_%REF\_%ALT' | bcftools view -Oz --threads 4 -i 'AF<=0.001 && MAC >=1 && F_MISSING<0.1 && F_PASS(

In [44]:
!{dx_command}

job-GPG9KY8JqBj5gK412JXv3f8G
